In [1]:
!pip install pandas

### Install the _kfmd_ library

In [2]:
# use local `kfmd` library
import sys
sys.path
sys.path.append('/home/jovyan/work/sdk/python/kfmd')

# To use the latest publish `kfmd` library, run
# !pip install kfmd

In [3]:
import pandas
import metadata

### Create a workspace

In [4]:
ws1 = metadata.Workspace(
    backend_url_prefix="127.0.0.1:8080",
    name="ws1",
    description="a workspace for testing",
    labels={"n1": "v1"})

### Create a run in a workspace

In [5]:
r = metadata.Run(
    workspace=ws1,
    name="first run",
    description="first run in ws_1",
)

### Log a data set

In [6]:
data_set = r.log(
        metadata.DataSet(
            description="an example data",
            name="mytable-dump",
            owner="owner@my-company.org",
            uri="file://path/to/dataset",
            version="v1.0.0",
            query="SELECT * FROM mytable"))
assert data_set.id

### Log a model

In [7]:
model = r.log(
    metadata.Model(
            name="MNIST",
            description="model to recognize handwritten digits",
            owner="someone@kubeflow.org",
            uri="gcs://my-bucket/mnist",
            model_type="neural network",
            training_framework={
                "name": "tensorflow",
                "version": "v1.0"
            },
            hyperparameters={
                "learning_rate": 0.5,
                "layers": [10, 3, 1],
                "early_stop": True
            },
            version="v0.0.1",
            labels={"mylabel": "l1"}))
assert model.id

### Log an evaluation of a model

In [8]:
metrics = r.log(
    metadata.Metrics(
            name="MNIST-evaluation",
            description="validating the MNIST model to recognize handwritten digits",
            owner="someone@kubeflow.org",
            uri="gcs://my-bucket/mnist-eval.csv",
            data_set_id=data_set.id,
            model_id=model.id,
            metrics_type=metadata.Metrics.VALIDATION,
            values={"accuracy": 0.95},
            labels={"mylabel": "l1"}))
assert metrics.id

### List all models in the workspace

In [9]:
pandas.DataFrame.from_dict(ws1.list(metadata.Model.ARTIFACT_TYPE_NAME))

,create_time,description,hyperparameters,id,labels,model_type,name,owner,run,training_framework,uri,version,workspace
0,2019-06-14T22:00:26.155444Z,model to recognize handwritten digits,"{'learning_rate': 0.5, 'layers': [10, 3, 1], '...",122,{'mylabel': 'l1'},neural network,MNIST,someone@kubeflow.org,first run,"{'name': 'tensorflow', 'version': 'v1.0'}",gcs://my-bucket/mnist,v0.0.1,ws1
1,2019-06-14T22:00:50.779346Z,model to recognize handwritten digits,"{'learning_rate': 0.5, 'layers': [10, 3, 1], '...",124,{'mylabel': 'l1'},neural network,MNIST,someone@kubeflow.org,first run,"{'name': 'tensorflow', 'version': 'v1.0'}",gcs://my-bucket/mnist,v0.0.1,ws1
2,2019-06-14T22:00:53.050369Z,model to recognize handwritten digits,"{'learning_rate': 0.5, 'layers': [10, 3, 1], '...",125,{'mylabel': 'l1'},neural network,MNIST,someone@kubeflow.org,first run,"{'name': 'tensorflow', 'version': 'v1.0'}",gcs://my-bucket/mnist,v0.0.1,ws1
3,2019-06-14T22:00:55.802075Z,model to recognize handwritten digits,"{'learning_rate': 0.5, 'layers': [10, 3, 1], '...",126,{'mylabel': 'l1'},neural network,MNIST,someone@kubeflow.org,first run,"{'name': 'tensorflow', 'version': 'v1.0'}",gcs://my-bucket/mnist,v0.0.1,ws1
4,2019-06-14T22:16:35.406927Z,model to recognize handwritten digits,"{'learning_rate': 0.5, 'layers': [10, 3, 1], '...",128,{'mylabel': 'l1'},neural network,MNIST,someone@kubeflow.org,first run,"{'name': 'tensorflow', 'version': 'v1.0'}",gcs://my-bucket/mnist,v0.0.1,ws1
5,2019-06-14T22:20:36.549267Z,model to recognize handwritten digits,"{'learning_rate': 0.5, 'layers': [10, 3, 1], '...",131,{'mylabel': 'l1'},neural network,MNIST,someone@kubeflow.org,first run,"{'name': 'tensorflow', 'version': 'v1.0'}",gcs://my-bucket/mnist,v0.0.1,ws1
6,2019-06-14T22:23:50.351408Z,model to recognize handwritten digits,"{'learning_rate': 0.5, 'layers': [10, 3, 1], '...",134,{'mylabel': 'l1'},neural network,MNIST,someone@kubeflow.org,first run,"{'name': 'tensorflow', 'version': 'v1.0'}",gcs://my-bucket/mnist,v0.0.1,ws1
